### Data Skipping and Z-Ordering
 With the release of Data Skipping in Delta Lake 1.2.0, column-level statistics like min/max are now available. Statistics are saved in the Delta Lake transaction log (DeltaLog) every time an `add` action is performed corresponding to adding a new Parquet file.
 By leveraging min-max ranges, Delta Lake is able to skip the Parquet files that are out of the range of the querying field values (Data Skipping). In order to make it effective, data can be clustered by Z-Order columns so that min-max ranges are narrow and, ideally, non-overlapping.

### Utility Functions

In [ ]:
import pandas as pd


def is_overlapped(a, b):
    return a[0] <= b[0] <= a[1] or b[0] <= a[0] <= b[1]


def get_stats(dt):
    lhs = dt.get_add_actions().to_pandas()[['min']].values.tolist()
    lhs = [x for r in lhs for x in r]
    lhs = pd.DataFrame.from_dict(lhs)
    rhs = dt.get_add_actions().to_pandas()[['max']].values.tolist()
    rhs = [x for r in rhs for x in r]
    rhs = pd.DataFrame.from_dict(rhs)
    return lhs, rhs


def get_num_overlapped(dt, intervals):
    ret, (lhs, rhs) = 0, get_stats(dt)
    for (_, min), (_, max) in zip(lhs.iterrows(), rhs.iterrows()):
        if all(map(lambda i: is_overlapped((i[1], i[2]), (min[i[0]], max[i[0]])), intervals)):
            ret += 1
    return ret

### Delta Table Structure

 You are wroking at a cyber security company. Your team collects traffic data which is created by an open source
network traffic analyzer. The schema is straightforward:

In [ ]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip


builder = SparkSession.builder.appName('CreateDeltaTables') \
    .config(
        'spark.jars.packages',
        'io.delta:delta-core_2.12:2.2.0') \
    .config(
        'spark.sql.extensions',
        'io.delta.sql.DeltaSparkSessionExtension') \
    .config(
        'spark.sql.catalog.spark_catalog',
        'org.apache.spark.sql.delta.catalog.DeltaCatalog')

spark = configure_spark_with_delta_pip(builder).getOrCreate()

security = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load('../../data/security.csv')
security.show(n=5, truncate=False)

The structure of the table is as follows:

In [ ]:
%%bash

tree -a ../../data/security-table

### Filter Clauses

 Suppose we are only interested in the traffic which satisfies the following conditions:

 - condition 1: 128.0.0.0 <= `src_ip` <= 191.255.255.255
 - condition 2: 1024 <= `src_port` <= 65535
 - condition 3: 128.0.0.0 <= `dst_ip` <= 191.255.255.255 and 1024 <= `dst_port` <= 65535

### Evaluation

 Let us define Skipping Effecticeness as follows:

`Skipping Effectiveness := # of filtered Parquet files / total # of Parquet files`

### Skipping Effectiveness

 Now, let's inspect the skipping effectiveness. Your end goal is likely to minimize the total amount of time spent on running these queries and the egress cost, but, for illustration purposes, let’s instead define our cost function as the total number of records scanned:

In [ ]:
from deltalake import DeltaTable

dt = DeltaTable('../../data/security-table')
cond_1 = get_num_overlapped(
    dt,
    [['src_ip', '128.0.0.0', '191.255.255.255']]
)
cond_2 = get_num_overlapped(
    dt,
    [['src_port', 1024, 65535]]
)
cond_3 = get_num_overlapped(
    dt,
    [['dst_ip', '128.0.0.0', '191.255.255.255'], ['dst_port', 1024, 65535]]
)

print('128.0.0.0 <= src_ip <= 191.255.255.255: ', 1 - cond_1 / 10)
print('1024 <= src_port <= 65535: ', 1 - cond_2 / 10)
print('128.0.0.0 <= dst_ip <= 191.255.255.255 and 1024 <= dst_port <= 65535: ', 1 - cond_3 / 10)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

left = np.array([1, 2, 3])
height = np.array([0.0, 0.0, 0.0])
label = ['#1', '#2', '#3']

plt.bar(left, height, tick_label=label, align='center')
plt.title('Skipping Effectiveness / No Optimization')
plt.xlabel('condition')
plt.ylabel('skipping effectiveness')
plt.ylim([0.0, 1.0])

### Partition by Range (Explicit Sorting)
 
 As our data is randomly generated and so there are no correlations. So let’s try explicitly sorting data before writing it.

In [ ]:
df = spark.read \
    .format('delta') \
    .load('../../data/security-table')
df.repartitionByRange(10, 'src_ip', 'src_port', 'dst_ip') \
    .write \
    .mode('overwrite') \
    .format('delta') \
    .save('../../data/security-table')

The structure of the table is as follows:

In [ ]:
%%bash

tree -a ../../data/security-table-part-by-range

### Skipping Effectiveness

 Now, let's inspect the skipping effectiveness. Your end goal is likely to minimize the total amount of time spent on running these queries and the egress cost, but, for illustration purposes, let’s instead define our cost function as the total number of records scanned:

In [ ]:
from deltalake import DeltaTable

dt = DeltaTable('../../data/security-table')
cond_1 = get_num_overlapped(
    dt,
    [['src_ip', '128.0.0.0', '191.255.255.255']]
)
cond_2 = get_num_overlapped(
    dt,
    [['src_port', 1024, 65535]]
)
cond_3 = get_num_overlapped(
    dt,
    [['dst_ip', '128.0.0.0', '191.255.255.255'], ['dst_port', 1024, 65535]]
)

print('128.0.0.0 <= src_ip <= 191.255.255.255: ', 1 - cond_1 / 10)
print('1024 <= src_port <= 65535: ', 1 - cond_2 / 10)
print('128.0.0.0 <= dst_ip <= 191.255.255.255 and 1024 <= dst_port <= 65535: ', 1 - cond_3 / 10)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

left = np.array([1, 2, 3])
height = np.array([0.6, 0.0, 0.0])
label = ['#1', '#2', '#3']

plt.bar(left, height, tick_label=label, align='center')
plt.title('Skipping Effectiveness / Explicit Sorting')
plt.xlabel('condition')
plt.ylabel('skipping effectiveness')
plt.ylim([0.0, 1.0])

### Z-Order Clustering
 
 Z-ordering is a technique to colocate related information in the same set of files. This optimization dramatically reduces the amount of data that Delta Table needs to read. 

In [ ]:
import delta

deltaTable = delta.DeltaTable.forPath(spark, '../../data/security-table')
spark.conf.set('spark.databricks.delta.optimize.maxFileSize', 1024*50)
deltaTable.optimize().executeZOrderBy('src_ip', 'src_port', 'dst_ip', 'dst_port')

The structure of the table is as follows:

In [ ]:
%%bash

tree -a ../../data/security-table

In [ ]:
from deltalake import DeltaTable

dt = DeltaTable('../../data/security-table')
cond_1 = get_num_overlapped(
    dt,
    [['src_ip', '128.0.0.0', '191.255.255.255']]
)
cond_2 = get_num_overlapped(
    dt,
    [['src_port', 1024, 65535]]
)
cond_3 = get_num_overlapped(
    dt,
    [['dst_ip', '128.0.0.0', '191.255.255.255'], ['dst_port', 1024, 65535]]
)

print('128.0.0.0 <= src_ip <= 191.255.255.255: ', 1 - cond_1 / 10)
print('1024 <= src_port <= 65535: ', 1 - cond_2 / 10)
print('128.0.0.0 <= dst_ip <= 191.255.255.255 and 1024 <= dst_port <= 65535: ', 1 - cond_3 / 10)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

left = np.array([1, 2, 3])
height = np.array([0.6, 0.4, 0.4])
label = ['#1', '#2', '#3']

plt.bar(left, height, tick_label=label, align='center')
plt.title('Skipping Effectiveness / Z-Order Clustering')
plt.xlabel('condition')
plt.ylabel('skipping effectiveness')
plt.ylim([0.0, 1.0])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

left = np.array([1, 2, 3])
height1 = np.array([0.0, 0.2, 0.2])
height2 = np.array([0.0, 0.0, 0.13])
height3 = np.array([0.0, 0.0, 0.13])
label = ['No Optimization', 'Explicit Sorting', 'Z-Order Clustering']

plt.bar(left, height1, tick_label=label, align="center")
plt.bar(left, height2, bottom=height1)
plt.bar(left, height3, bottom=height1 + height2)
plt.plot(left, [0.0, 0.2, 0.46], color='red')
plt.scatter(left, [0.0, 0.2, 0.46], color='red')
plt.title("Skipping Effectiveness")
plt.ylabel("skipping effectiveness")
plt.ylim([0.0, 1.0])